# Splash dynamics on a cone surface



## Documentation and Imports


Created on 27-07-21

Author: Valentin Laplaud

Formulas and graphs describing the dynamics of a drop splashing on a cone

In [ ]:
# plotting stuff
import matplotlib as mpl
mpl.use('TkAgg')
%matplotlib inline

COLOR = 'white'
COLOR2 = 'black'

mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams['axes.edgecolor'] = COLOR

mpl.rcParams["figure.facecolor"] = COLOR2
mpl.rcParams["axes.facecolor"] = COLOR2
mpl.rcParams["savefig.facecolor"] = COLOR2

import matplotlib.pyplot as plt
from cycler import cycler


# numbers handling
import numpy as np
from numpy import pi
import pandas as pd

# signal processing 
from scipy.signal import savgol_filter, correlate, correlation_lags
from scipy.interpolate import interp1d

# images handling
from skimage import io
from skimage.filters import threshold_mean, gaussian, laplace, sobel
from skimage.measure import label, regionprops, regionprops_table
from skimage.util import invert
from skimage.morphology import binary_opening, binary_closing, remove_small_holes,binary_erosion
from skimage.color import rgb2gray, rgb2hsv
from skimage.segmentation import active_contour, morphological_geodesic_active_contour,morphological_chan_vese, checkerboard_level_set, inverse_gaussian_gradient
import cv2 as cv
import pims
import trackpy as tp

# to hide known warnings
import warnings
warnings.filterwarnings("ignore")

# General system functions
import os
import shutil
import sys

import time

# my functions
sys.path.append('../PythonFuncs/')
import VallapFunc_DP as vf


## Mega cone experiments

### Tracking 

In [ ]:
def Track(P,StackList,SD,Scale,FPS):

    SD = SD.loc[StackList]
    
    TD = pd.DataFrame(data=None)
    
    if not os.path.exists(P + '\\Tracking'):
            os.mkdir(P + '\\Tracking') # create global folder 
    
    if not os.path.exists(P + '\\Trajectories'):
            os.mkdir(P + '\\Trajectories') # create global folder  
    
    for s in StackList:
        
        if not os.path.exists(P + '\\Tracking\\' + s):
            os.mkdir(P + '\\Tracking\\' + s) # create folder 
        
        if not os.path.exists(P + '\\Trajectories\\' + s):
            os.mkdir(P + '\\Trajectories\\' + s) # create folder 
        
        Stack = io.imread(P + '\\' + s + '_Processed_Cut.tif') # get the tiff stack
        
        print('Tracking for video : ' + s)
        # Drop position
        Xd = SD.loc[s,'DropXc']
        Yd = SD.loc[s,'DropYc']
        Rd = SD.loc[s,'DropDiam']/2*Scale
        # Target position
        Xt = SD.loc[s,'TargetXc']
        Yt = SD.loc[s,'TargetYc']
        Rt = SD.loc[s,'TargetDiam']/2*Scale
        
        # Tracking
        frames = pims.open(P + '\\' + s + '_Processed_Cut.tif')
        tp.quiet()
        f = tp.batch(frames, 11, invert=False, minmass=500) # 15px size estimate
        f['CenterDist'] = vf.dist(f['x'].values,f['y'].values,Xt,Yt)
        f = f.loc[f['CenterDist']<Rt]

  #      for i in range(len(frames)):
  #          fig, ax = plt.subplots(dpi=250)
  #          fig.suptitle(s + '_img' + str(i))
   #         tp.annotate(f[f['frame']==i], frames[i])
    #        fig.savefig(P + '\\Tracking\\' + s + '\\' + str(i) + '.tif')
    #        plt.close()

#        @tp.predict.predictor
#        def predict(t1, particle):
#            velocity = 3*(np.array(((particle.pos[0]-Xd),(particle.pos[1]-Yd)))
#                          /np.sqrt(np.square(particle.pos[0]-Xd)+np.square(particle.pos[1]-Yd))) # Going away from target
#            return particle.pos + velocity * (t1 - particle.t)
        
        t = tp.link(f, 5, memory=1) # px max from the prediction, images memory 
        
        
        t = tp.filter_stubs(t, 15) # remove trajectories with less than 10 images (1 ms)
             
        particleList = np.unique(t['particle'].values)
        
        for p in particleList:
            trajP = t[t['particle'] == p]
            X = trajP['x'].values
            Y = trajP['y'].values
            length = vf.dist(X[len(X)-1],Y[len(X)-1],
                             X[0],Y[0])
            if length>30:
                t.loc[t['particle'] ==p,'lengthOK'] = True
            else:
                t.loc[t['particle'] ==p,'lengthOK'] = False
                
        
        t['Splash'] = s
        
        OffC = SD.loc[s,'OffCentering_pcDropDiam']
       
        
        TD = TD.append(t[t['lengthOK']], ignore_index=True)
        TD.loc[TD['Splash'] == s,'OffC'] = OffC
        
        fig,ax = plt.subplots(dpi=300)
        absc = np.linspace(0,2*pi,1000)
        ax.plot(Xt,Yt,'.g',ms=2)
        ax.plot(Xt+Rt*np.cos(absc),Yt+Rt*np.sin(absc),'--g',lw=0.5)
        ax.plot(Xd,Yd,'.r',ms=2)
        ax.plot(Xd+Rd*np.cos(absc),Yd+Rd*np.sin(absc),'--r',lw=0.5)
        tp.plot_traj(t[t['lengthOK']],ax = ax,plot_style={"linewidth": 0.6, "linestyle":'--'})
        ax.set_aspect('equal', adjustable='box')
        fig.suptitle(s)
        fig.savefig(P + '\\Trajectories\\' + s + '.tif')
        #plt.close()
        plt.show()
        
        for i in range(len(Stack)-1):
            fig0, ax0 = plt.subplots(dpi=250)
            plt.imshow(Stack[i],cmap='gray')
            xl = ax0.get_xlim()
            yl = ax0.get_ylim()
            if not t.loc[(t['frame']<i) & t['lengthOK']].empty:
                tp.plot_traj(t.loc[(t['frame']<i) & t['lengthOK']], ax = ax0)
            ax0.set_xlim(xl)
            ax0.set_ylim(yl)
            fig0.savefig(P + '\\Trajectories\\' + s + '\\' + str(i) + '.tif')
            plt.close()
        

    return(SD,TD)
        

### Plotting tracks

In [ ]:
def plotTrackModel(SD,TD,StackList,ConeAngles,P):
    
    for a,s in zip(ConeAngles,StackList):
        
        # Positions of cone center
        Xc = SD.loc[s,'TargetXc']
        Yc = SD.loc[s,'TargetYc']
        
        # radius of the original circle, and angle of the cone
        Rcircle = SD.loc[s,'TargetDiam']/2*Scale210721/np.sin(a)
        Alpha = a
        
        # Position of drop impact on the cone
        Xd = SD.loc[s,'DropXc'] - Xc
        Yd = SD.loc[s,'DropYc'] - Yc
        
        # Position of drop impact on the cone in cilyndrical coordinate centered on the cone
        Ad,Rd = vf.ToCirc(Xd,Yd)

        # position of drop impact on the original circle (angle doesn't change as the impact is the 0° reference)
        Xi,Yi = vf.ToCart(Ad,Rd/np.sin(a))
        
        # # position of drop impact on the original circle in carthésian coordinates
        Xi = Xi 
        Yi = Yi 
        
        # Trajectories for stack s
        Trajs = TD[TD['Splash'] == s]
        
        # list of detected particles
        partList = np.unique(Trajs['particle'].values)
        
        # angles of the trajectories starting point
        angles = [];   
        
        # figure or the representations
        fig1, [ax1,ax2] = plt.subplots(ncols=2, dpi=300)
        
        tx = np.linspace(0,2*pi,100)
        
        ax1.set_title('Trajectories on the cone')
        ax1.set_aspect('equal', adjustable='box') 
        ax1.plot(0,0,'g*')
        ax1.plot(Rcircle*np.sin(a)*np.cos(tx),Rcircle*np.sin(a)*np.sin(tx),'g-') 
        ax1.plot(Xd,Yd,'r*')
        
        ax2.set_title('Trajectories on the flat unfolded circle')
        ax2.set_aspect('equal', adjustable='box') 
        ax2.plot(0,0,'g*')
        ax2.plot(Rcircle*np.cos(tx),Rcircle*np.sin(tx),'g-') 
        ax2.plot(Xi,Yi,'r*')
        
        for p in partList:       
            
            # trajectory on the cone for particle p
            Xp = Trajs.loc[Trajs['particle'] == p,'x'].values - Xc
            Yp = Trajs.loc[Trajs['particle'] == p,'y'].values - Yc 
            
            # plotting
            ax1.plot(Xp,Yp,'--',lw=1)     
            
            # Cylindrical coordinate of the trajectory on the cone
            A,R = vf.ToCirc(Xp,Yp)
            
            
            # coordinates on the original circle
            Ashift = A-Ad
            Ashift[Ashift>pi] = np.mod(Ashift[Ashift>pi],pi) - pi
            
            Acircle = Ashift*np.sin(a) + Ad

            Xpc,Ypc = vf.ToCart(Acircle,R/np.sin(a))
            
            # plotting
            ax2.plot(Xpc,Ypc,'--',lw=1) 
            
            # Angle of this trajectory            
            angles = np.append(angles,vf.ToCirc(Xpc[0]-Xi,
                                                Ypc[0]-Xi,angle='rad')[0])           
        
        fig1.savefig(P + '\\' + s + '_CircleTraj.tif')
        #plt.close(fig1)
        
        fig, ax = ComputeTransfo(Xc,Yc,Rcircle,Alpha,Xi,Yi,angles)
        fig.suptitle('Trajectories on the cone for ' + s)
        
        tp.plot_traj(Trajs,ax = ax[3],plot_style={"linewidth": 0.6, "linestyle":'--'})
        
        fig.tight_layout()
        fig.savefig(P + '\\' + s + '_Model.tif')
        
        return
        
    
    

## Run tracking

### Data details

In [ ]:
# Pixels per microns
Scale210721 = 10.89 
# Frames per second
FPS210721 = 10000 
# Paths to data
P210721 = r'D:\Users\laplaud\Desktop\PostDoc\Data\SplashCups\2021.07.21'
# List of videos to analyze


        
StackList210721 = [ 'MCC0_30(N)-1','MCC0_30(N)-2','MCC0_30(N)-3','MCC0_30(N)-4','MCC0_30(N)-5',
                  'MCC0_45-1','MCC0_45-2','MCC0_45-3','MCC0_45-4',
                   'MCC0_45-5','MCC0_45-6','MCC0_45-7','MCC0_45-8','MCC0_45-9',
                  "MCC0_30-1","MCC0_30-2","MCC0_30-3","MCC0_30-4","MCC0_30-5","MCC0_45(S)-1",
                   "MCC0_45(S)-2","MCC0_45(S)-3","MCC0_45(S)-4","MCC0_45(S)-5","MCC0_60-1","MCC0_60-2",
                   "MCC0_60-3","MCC0_60-4","MCC0_60-5","MCC0_60-6","MCC0_60-7"] 

# angles of the cone with the vertical axis
ConeAngles210721 = np.array([60,60,60,60,60,45,45,45,45,45,45,45,45,45,60,60,
                    60,60,60,45,45,45,45,45,30,30,30,30,30,30,30])/360*2*pi



print('Data choice made.')

### Loading

In [ ]:
SplashData210721 = pd.read_csv(P210721 + '\\SplashData210721_Circles.csv',index_col = 'Ind')

### Compute trajecories

In [ ]:
SplashData210721, TrackData210721 = Track(P210721,StackList210721,SplashData210721,Scale210721,FPS210721)

### Compare with theory

In [ ]:
plotTrackModel(SplashData210721,TrackData210721,StackList210721,ConeAngles210721,r'D:\Users\laplaud\Desktop\PostDoc\Data\SplashCups\2021.07.21\Trajectories')

In [ ]:
expes = [ 'MCC0_30(N)-1','MCC0_30(N)-2','MCC0_30(N)-3','MCC0_30(N)-4','MCC0_30(N)-5',
            "MCC0_45(S)-1","MCC0_45(S)-2","MCC0_45(S)-3","MCC0_45(S)-4","MCC0_45(S)-5",
            "MCC0_45-1","MCC0_45-6","MCC0_45-7","MCC0_45-8","MCC0_45-9"]


for expe in expes:
    SDtest = SplashData210721.loc[expe]

    # 'MCC0_30(N)-2' 'MCC0_60-1' 

    Alpha = pi/4
    Xc = SDtest['TargetXc']
    Yc = SDtest['TargetYc']
    Rcircle = SDtest['TargetDiam']/2*Scale210721/np.cos(Alpha)
    Xi = SDtest['DropXc']
    Yi = SDtest['DropYc']

    Ai,Ri = vf.ToCirc(Xi-Xc,Yi-Yc)
    Xi,Yi = vf.ToCart(Ai,Ri/np.cos(Alpha))
    Xi = Xi + Xc
    Yi = Yi + Yc

    Img = io.imread(r'D:\Users\laplaud\Desktop\PostDoc\Data\SplashCups\2021.07.21\Traj\\' + expe + '_Traj.tif')

    angles = np.linspace(0,2*pi,30) + vf.ToCirc(Xc-Xi,Yc-Yi)[0] - pi
    T = '45° Cone from a circle'
    P = r'D:\Users\laplaud\Desktop\PostDoc\Data\SplashCups\2021.07.21'

    fig, ax = ComputeTransfo(Xc,Yc,Rcircle,Alpha,Xi,Yi,angles,animation=False,title = T, path = P)

    ax[3].imshow(Img,cmap='gray')

    fig.savefig(P + '\\Traj\\' + expe + '_WithModel.tif')